# Betas

This notebook estimates (single and multi-) index models.

## Load Packages and Extra Functions

In [1]:
MyModulePath = joinpath(pwd(),"src")     #add /src to module path
!in(MyModulePath,LOAD_PATH) && push!(LOAD_PATH,MyModulePath)
using FinEcmt_OLS

In [2]:
#=
include(joinpath(pwd(),"src","FinEcmt_OLS.jl"))
using .FinEcmt_OLS
=#

In [3]:
using DelimitedFiles, LinearAlgebra

## Loading Data

We load data from two data files: for the returns on Fama-French equity factors and then also for the 25 Fama-French portfolios. To keep the output simple (easy to display...), we use only 5 of those portfolios.

In [4]:
x    = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)
Rme  = x[:,2]                #market excess return
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate

x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1,7,13,19,25]]           #use just 5 assets to make the printing easier

(T,n) = size(Re)                    #no. obs and  no. test assets

(388, 5)

# Estimating a Single-Index Model

for `n` returns, by looping. (As an alternative, `OlsSure()` can do all regressions in one call.)

In [5]:
x = [ones(T) Rme]                   #regressors
K = size(x,2)                       #no. regressors      

(β,tstat) = (fill(NaN,K,n),fill(NaN,K,n))    #to fill with results 
for i in 1:n
    (b,_,_,V,) = OlsGM(Re[:,i],x)
    β[:,i]     = b
    stdErr     = sqrt.(diag(V))
    tstat[:,i] = b./stdErr
end    

colNames = [string("asset ",i) for i=1:n]
rowNames = ["α","β on Rme"]

printblue("coeffs for $n assets, from OLS of Re on constant and Rme:")
printmat(β;colNames,rowNames)
printblue("t-stats for $n assets, from OLS of Re on constant and Rme:")
printmat(tstat;colNames,rowNames)

coeffs for 5 assets, from OLS of Re on constant and Rme:
           asset 1   asset 2   asset 3   asset 4   asset 5
α           -0.504     0.153     0.305     0.279     0.336
β on Rme     1.341     1.169     0.994     0.943     0.849

t-stats for 5 assets, from OLS of Re on constant and Rme:
           asset 1   asset 2   asset 3   asset 4   asset 5
α           -1.656     1.031     2.471     2.163     2.073
β on Rme    20.427    36.534    37.298    33.848    24.279



# Estimating a Multi-Index Model

A multi-index model is based on 

$R_{it} =a_{i}+b_{i}^{\prime}I_{t}+\varepsilon_{it}$,

where $b_{i}$ is a $K$-vector of slope coefficients.

In [6]:
x = [ones(T) Rme RSMB RHML]               #regressors
K = size(x,2)                       #no. regressors      

(β,tstat) = (fill(NaN,K,n),fill(NaN,K,n))    #to fill with results 
for i in 1:n
    (b,_,_,V,) = OlsGM(Re[:,i],x)
    β[:,i]     = b
    stdErr     = sqrt.(diag(V))
    tstat[:,i] = b./stdErr
end    

colNames = [string("asset ",i) for i=1:n]
rowNames = ["α","β on Rme", "β on RSMB", "β on RHML"]

printblue("coeffs for $n assets, from OLS of Re on constant and Rme:")
printmat(β;colNames,rowNames)
printblue("t-stats for $n assets, from OLS of Re on constant and Rme:")
printmat(tstat;colNames,rowNames)

coeffs for 5 assets, from OLS of Re on constant and Rme:
            asset 1   asset 2   asset 3   asset 4   asset 5
α            -0.513    -0.006     0.030    -0.020    -0.015
β on Rme      1.070     1.080     1.035     1.056     1.041
β on RSMB     1.264     0.768     0.437     0.153    -0.088
β on RHML    -0.278     0.160     0.487     0.603     0.770

t-stats for 5 assets, from OLS of Re on constant and Rme:
            asset 1   asset 2   asset 3   asset 4   asset 5
α            -2.306    -0.066     0.328    -0.206    -0.133
β on Rme     21.091    50.113    50.250    48.060    41.247
β on RSMB    17.068    24.418    14.534     4.757    -2.380
β on RHML    -3.629     4.928    15.665    18.186    20.227

